In [4]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [5]:
from TexSoup import TexSoup
import re

def find_sections(tex):
    return [match.group(1) for match in re.finditer("\\\section\{(.*)\}", tex)]


def extract_content(node, name):
    if node:
        return ((name, node.contents[0]))


def section_split(tex):
    """returns: List of tuples (section_name, section_text)"""
    doc_text = tex
    sections = []
    # parsing with TexSoup
    # TODO: get rid of this and use regex only (brittle)
    try:
        soup = TexSoup(tex, tolerance=1)
        if title:=extract_content(soup.title, "title"):
            sections.append(title)
        if abstract:=extract_content(soup.abstract, "abstract"):
            sections.append(abstract)
        sections.append(("tables", "\n".join([str(node) for node in soup.find_all("table")]))) if soup.find_all("table") else None
    except:
        pass # could not parse tex

    # extract latex sections and corresponding text
    prev_section = "pre"

    for section in find_sections(tex):
        section_text, doc_text = doc_text.split(f"\\section{{{section}}}", 1)
        sections.append((prev_section, section_text))
        prev_section = section
    sections.append((prev_section, doc_text))
    return sections
    # except:
    #     return (("full", tex))
    


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\urban\AppData\Local\Temp\ipykernel_145896\3670686320.py:5: SyntaxWarning: invalid escape sequence '\s'
  return [match.group(1) for match in re.finditer("\\\section\{(.*)\}", tex)]


In [1]:
import ollama


def pass_to_ollama(prompt, model):
    try:
        res = ollama.generate(model=model, prompt=prompt, options={"temperature": 0})
        return res["response"]
    except Exception as ex:
        print(ex)
        return f"ollama error: {ex}"


In [7]:
import json

from src.dataset import UNANSWERABLE


# prompt template
def extract_tdms(tex, few_shot=True):
    if few_shot:
        return f"""If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.
        
        Text: {tex}
        
        Return the tasks, datasets, metrics and scores as reported in the text in a JSON array. Do not include precision information in the reported score.
        Here the formating structure of the JSON. Please use exactly this formating in your answer.
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        Lets make an example for you: Template-Based Automatic Search of Compact Semantic Segmentation Architectures... One discovered architecture achieves 63.2% mean IoU on CamVid and 67.8% on CityScapes having only 270K parameters... evaluation.
        
        The expected answer of you is:
        [
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CamVid", "Metric": Mean IoU", "Score": "63.2"}}, 
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CityScapes", "Metric": Mean IoU", "Score": "67.8"}}
        ]
        
        """
    else:
        return f"""If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.
        
        Text: {tex}
        
        Return the tasks, datasets, metrics and scores as reported in the text in a JSON array. Do not include precision information in the reported score.
        Here the formating structure of the JSON. Please use exactly this formating in your answer.
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        """

zero_shot_template = lambda tex: extract_tdms(tex, False)
few_shot_template = lambda tex: extract_tdms(tex, True)

def convert_tdms_to_tuple(model_output_parsed):
    tuples = []
    for item in model_output_parsed:
        try:
            t = ((item["Task"], item["Dataset"],item["Metric"],item["Score"]))
            tuples.append(t)
        except:
            # parse error, ignore instance
            pass
    return tuples

def format_tdms(tuples):
    """make unique, format as string"""
    unique = set(tuples)
    dicts = [{"LEADERBOARD": {
        "Task": t,
        "Dataset":d,
        "Metric":m,
        "Score":s
    }} for t,d,m,s in unique]
    return str(dicts)


def section_wise_extraction(tex, prompt_template, model):
    """takes a tex file, returns an annotation"""
    found_tdms = []
    sections = section_split(str(tex))
    for section_name, section_text in sections:
        prompt = prompt_template
        response = pass_to_ollama(prompt, model)

        try:
            response = json.loads(response)
            parsed = convert_tdms_to_tuple(response)
            found_tdms= [*found_tdms, *parsed]
        except:
            pass # no tuples found in section

    # print(found_tdms)
    if not found_tdms:
        annotation = UNANSWERABLE # found_tdms are empty -> unanswerable
    else:
        # dedupe and format
        annotation = format_tdms(found_tdms)
    return annotation

In [3]:
from src.dataset import PATH
from src.experiment_runner import run
from src.helper import get_file_content_as_string
from src.helper import model_name_existing

#Create Model
model_list = ollama.list()
model_name="llama3:8b"
if model_name_existing(model_list, model_name):
    model = model_name
else:
    ollama.create(model="llama3:8b", modelfile=get_file_content_as_string("src/modelfile_llama3_8b"))

# Run on Test
df_test = run(lambda tex: section_wise_extraction(tex, zero_shot_template, model), PATH.TEST, "llama3_70b_zeroshot")
df_test = run(lambda tex: section_wise_extraction(tex, few_shot_template, model), PATH.TEST, "llama3_70b_fewshot")


KeyboardInterrupt: 